## Pirate Analyzer

The aim of this notebook is to analyze an individual pirate, or the top pirates in a ship. Mainly comparing hyperparameters and looking at the output of the pirate model on some test images



In [ ]:
import os
import sys
import logging
import json
# Allow for importing Pirate class
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, '..'))
from src.dataset import Dataset, plot_images
from src.pirate import Pirate
from src.ship import Ship
from src.config import *

# Define logger
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

### Individual Pirate Analysis

This section looks at a single pirate. You will need to provide the dna for a given pirate. The easiest way to do this is to run

`sqlitebrowser local/ships/yourshipname.db`

scroll through the list of pirates and pick one.


In [ ]:
# We need the dna of the pirate in question
DNA = '60e43a29-2c95-4342-8b5d-b86015a96ac3'

# Note that initializing a pirate this way means you lose the name/stats :(
pirate = Pirate(dna=DNA)

# Make a quick test dataset
test_path = os.path.join(DATA_DIR, '0312cleaner')
test = Dataset.from_path(test_path, max_n = 100)
images, true_labels = test.load_data_sample(n = 18)

# Get the pirate action associated with each image
pred_labels = []
for image in images:
    label = pirate.act(image)
    pred_labels.append(label)
    
# Plot the images along with true and predicted labels
plot_images(images, true_labels, pred_label=pred_labels)

# Print out the hyperparameters for this model
print('Hyperparams %s' % json.dumps(pirate.description(), indent=4))

### Ship Analysis

This section will analyze the pirates within an entire ship. The main interest here is to find the hyperparameters that seem to be producing the best pirates. 

In [ ]:
SHIP_NAME = 'The Black Pearl'
N_BEST_PIRATES = 6

# Get the best pirates
with Ship(ship_name=SHIP_NAME) as ship:
    pirates = ship.get_best_pirates(N_BEST_PIRATES)

# Aggregate dictionary of hyperparameters:
agg_params = {}
for pirate in pirates:
    for key, value in pirate.description().items():
        # Add the pirate hyperparameter to aggregate
        if key not in agg_params.keys():
            agg_params[key] = []
        # Add parameter value
        agg_params[key].append(value)

# Print out the hyperparameters for this model
print('Aggregate Hyperparameters %s' % json.dumps(agg_params, indent=4))